# Pre-configurate

In [1]:
import numpy as np
import pandas as pd
# from scipy.spatial import KDTree
from sklearn import neighbors
import matplotlib.pyplot as plt 
import pyproj
import os
from tqdm import tqdm
import warnings

In [5]:
warnings.filterwarnings('ignore')

In [2]:
bp_file=r'E:/phD_career/活儿/173/data/base_points/ObjResidual.txt'
SLACP_path=r'E:/phD_career/活儿/173/data/ATLAS/2020/'
output_file=r'E:/phD_career/活儿/173/data/base_points/ObjResidual_adjacent_cps.txt'

geosrs="epsg:4326"
prosrs="epsg:32651"
level_threshold=3
slope_threshold=0.01
search_type='range'
search_range=5e5
box=[116,119,36,39]

# Read base points

In [7]:
def read_bp_file(bp_file):
    '''
    load base points file
    input:
    bp_file: base point file
    output:
    bp_digital: control points in dataframe format [longitute	lattitute	level]
    '''
    with open(bp_file,'r') as f:
        bp=f.read()
        bp_list=bp.split('\n')
        bp_digital=[each.split('  ') for each in bp_list[3:-1]]
        bp_digital=np.array([list(filter(None,each)) for each in bp_digital])
        bp_digital=bp_digital.astype('float')
    return bp_digital[:,1:5]

base_points_arr=read_bp_file(bp_file)

# Extract adjacent control points

In [8]:
def read_cp_file(cp_file):
    '''
    load control points file
    input:
    cp_file: the control point file
    output:
    cp: control points in dataframe format, [index	longitute	lattitute	level	height level	terrain slope]
    '''
    cp = pd.read_csv(cp_file, sep='\t', header=0)
    return cp

def project_points(points,geosrs,prosrs):
    '''
    geolocating the current point or points (in dataframe format)
    input:
    points: current point or points
    geosrs: geo-reference system
    prosrs: projection refrence system
    output:
    control points in dataframe format [longitute	lattitute	level]
    '''
    p1 = pyproj.Proj(init=geosrs)#WGS84
    p2 = pyproj.Proj(init=prosrs)
    xprj, yprj = pyproj.transform(p1, p2,points.iloc[:,1:3], points.iloc[:,1:3])
    return xprj,yprj

def extract_in_loop(base_points_arr,cp_path,geosrs,prosrs,level_threshold,slope_threshold,box=None,search_type=None,search_range=None,search_amount=None):
    '''
    extracting adjacent points in iterations
    input:
    bp_file: base point(center of the circle) file
    cp_path: control point file
    geosrs: geolocation refrence system
    prosrs: projection refrence system
    level_threshold: the altimetry accuracy threshold, control point that is HIGHER than this will be excluded
    slope_threshold: the slope threshold, control point whose slope is HIGHER than this will be excluded
    search_type: 'range': search by range, 'amount': search by amount of adjacent points
    search_range: search range, in meter
    search_amount: amount of adjacent points
    output:
    nearby_cp_filtered: extracted adjacent points, further filtered under slope threshold and accuracy threshold
    '''
    filtered_indices=[]

    base_points=pd.DataFrame(base_points_arr,columns=['pointID','longitute','latitute','level'])
    xprj,yprj=project_points(base_points.iloc[:,1:3],geosrs,prosrs)
    base_points_prjd=base_points
    base_points_prjd.iloc[:,1]=xprj 
    base_points_prjd.iloc[:,2]=yprj

    cp_dirs=os.listdir(cp_path)
    nearby_cp_filtered = pd.DataFrame(columns=['bp_index', 'nearby_cp'])

    for index in tqdm(range(0,base_points_prjd.shape[0]),desc='extracting adjacent control points'):
        nearby_cp_tmp=np.empty(shape=[0,4])
        for cp_dir in cp_dirs:
            cp_sub_dir=cp_path+cp_dir+'/data/'
            cp_files=os.listdir(cp_sub_dir)
            for cp_file in cp_files:
                control_points=read_cp_file(cp_sub_dir+cp_file)
                if (box[0]<=control_points.loc[:,'lon']).any() and (control_points.loc[:,'lon']<=box[1]).any() and (box[2]<=control_points.loc[:,'lat']).any() and (control_points.loc[:,'lat']<=box[3]).any():
                    xprj,yprj=project_points(control_points.iloc[:,1:3],geosrs,prosrs)
                    control_points_prjd=control_points
                    control_points_prjd.iloc[:,1]=xprj
                    control_points_prjd.iloc[:,2]=yprj
                    control_points_tree=neighbors.KDTree(control_points_prjd.iloc[:,1:3].values)
                    filtered_indices=[]
                    if search_type=='range':
                        nearby_cp_indices=control_points_tree.query_radius(base_points_prjd.iloc[index,1:3].values.reshape(1,-1),r=search_range)
                    nearby_cp_indices=np.concatenate(nearby_cp_indices)
                    for index_ in nearby_cp_indices:
                        if control_points_prjd.iloc[index_,4]<=level_threshold and np.fabs(control_points_prjd.iloc[index_,5])<=slope_threshold:
                            filtered_indices.append(index_)
                        else: continue
                    if filtered_indices:
                        nearby_cp_tmp=np.vstack(control_points_prjd.iloc[filtered_indices,1:-1].values)
                else:
                    continue
        nearby_cp_filtered.loc[len(nearby_cp_filtered),['bp_index','nearby_cp']]=[index,nearby_cp_tmp]
    return nearby_cp_filtered

extracted_points=extract_in_loop(base_points_arr,SLACP_path,geosrs,prosrs,level_threshold,slope_threshold,box,search_type,search_range)

extracting adjacent control points:   0%|          | 0/27316 [00:00<?, ?it/s]

extracting adjacent control points: 100%|██████████| 27316/27316 [1:23:03<00:00,  5.48it/s]  


# Save to txt file

In [9]:
def output_cps(base_points,nearby_cp,output_file,type=None):
    if type=="SAR":
        with open(output_file,'w+') as f:
            amount=len(base_points)
            f.write(str(amount)+'\n')
            for index in tqdm(range(amount),desc='writing to file'):
                f.write('Begin\t%s\t%.4f\t%.4f\t%.4f\n' %(str(base_points[index,0]),base_points[index,1],base_points[index,2],base_points[index,3]))
                nearby_cp_tmp=nearby_cp.loc[index,'nearby_cp']
                for each in nearby_cp_tmp:
                    f.write('%.4f\t%.4f\t%.4f\n' %(each[0],each[1],each[2]))
                f.write('End\n')   
                f.write('\n')
    elif type=='XQsoftware':
        with open(output_file) as f:
            pass

output_cps(base_points_arr,extracted_points,output_file,type='SAR')

writing to file: 100%|██████████| 27316/27316 [00:12<00:00, 2182.75it/s]
